Usecase test 2: Which articles contain data regarding the gene NRXN2

1. Query to find all articles containing reference to NRXN2 in the supporting data - HGNC:8009

In [2]:
import rdflib

#filename = "test_graph.nt"
filename = "cleaned_maingraph.nt"
g = rdflib.Graph()
g.parse(filename, format="nt")


<Graph identifier=Nbbd4b17dde184f3ca0e3568854213721 (<class 'rdflib.graph.Graph'>)>

In [10]:
gene_query = """
    PREFIX EDAM: <http://edamontology.org/>
    PREFIX RDF: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX DCT: <http://purl.org/dc/terms/>
    PREFIX BIOLINK: <https://w3id.org/biolink/vocab/>
    PREFIX ENSEMBL: <http://identifiers.org/ensembl/>
    PREFIX NCBIGENE: <http://identifiers.org/ncbigene/>
    PREFIX XSD: <http://www.w3.org/2001/XMLSchema#>

    SELECT ?root_subject (COUNT(DISTINCT ?root_subject) AS ?count)
        WHERE {
            FILTER(STRSTARTS(STR(?root_subject), "https://pubmed.ncbi.nlm.nih.gov/")) .
            ?root_subject ?p1 ?intermediate1 .
            ?intermediate1 ?p2 ?intermediate2 .
            ?intermediate2 BIOLINK:Gene ?gene .
            ?intermediate2 EDAM:data_1669 ?pvalue .
            FILTER(?pvalue < 0.05) .
            #?intermediate2 EDAM:data_3754 ?logvalue .
            #FILTER(DATATYPE(?logvalue > 1.0)) .
            FILTER(CONTAINS(STR(?gene), "HGNC:8009"))
        }
"""
#returns count of distinct articles with the term HGNC:8009 as a connected gene, with significant gene values
result = g.query(gene_query)

for row in result:
    root_subject = row[0]    # ?root_subject
    count = row[1]           # ?count

    print(f"Root Subject: {root_subject}, Count: {count}")

Root Subject: https://pubmed.ncbi.nlm.nih.gov/36323788, Count: 6


In [ ]:
#repeating with less common autism protein - HSPB1, HGNC:5246

heatprotein_query = """
    PREFIX EDAM: <http://edamontology.org/>
    PREFIX RDF: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX DCT: <http://purl.org/dc/terms/>
    PREFIX BIOLINK: <https://w3id.org/biolink/vocab/>
    PREFIX ENSEMBL: <http://identifiers.org/ensembl/>
    PREFIX NCBIGENE: <http://identifiers.org/ncbigene/>
    PREFIX XSD: <http://www.w3.org/2001/XMLSchema#>

    SELECT ?gene ?root_subject
    WHERE {
        ?root_subject ?p ?o
        FILTER(STRSTARTS(STR(?root_subject), "https://pubmed.ncbi.nlm.nih.gov/"))
        ?root_subject ?p1 ?intermediate1 .
        ?intermediate1 ?p2 ?intermediate2 .
        ?intermediate2 BIOLINK:Gene ?gene .
        FILTER(CONTAINS(STR(?gene), "HGNC:5246")) .
        ?intermediate2 EDAM:data_1669 ?pvalue .
        FILTER(DATATYPE(?pvalue) = XSD:double && ?pvalue < 0.05) .
        ?intermediate2 EDAM:data_3754 ?logvalue .
        FILTER(DATATYPE(?logvalue) = XSD:double && (?logvalue > 0.5 || ?logvalue < -0.5))
    }
    GROUP BY ?gene
    ORDER BY DESC(?count)
    LIMIT 50
"""
results = g.query(heatprotein_query)
print("Gene | article")
print("-----|------")
for row in results:
    gene = str(row['gene'])
    count = int(row['root_subject'].toPython()) 
    print(f"{gene} | {count}")